In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install snntorch

In [3]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import matthews_corrcoef
from torchvision import models
import snntorch as snn
from snntorch import surrogate
from snntorch import utils
import gc
import matplotlib.pyplot as plt

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "base_dir": "/content/drive/MyDrive/glacier/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_DeepLab_ResNet50",

    "model_type": "CNN",   # Run CNN then SNN

    # PARAMS
    "time_steps": 6,
    "batch_size": 2,       # BS=2 for ResNet50 + DeepLab (Heavy!)
    "lr": 1e-4,
    "epochs": 45,
    "beta": 0.9,
    "threshold": 0.5,
    "slope": 25,

    "num_workers": 2,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

os.makedirs(CONFIG['project_dir'], exist_ok=True)
torch.cuda.empty_cache()
gc.collect()

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

# ==========================================
# 2. DATASET
# ==========================================
class GlacierDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = Path(base_dir)
        self.band_dirs = [self.base_dir / f"Band{i}" for i in range(1, 6)]
        self.label_dir = self.base_dir / "labels"
        if not self.band_dirs[0].exists(): raise FileNotFoundError("Check Drive Path!")
        self.ids = sorted([p.stem for p in self.band_dirs[0].glob("*.tif")])
        self.transform = transform

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        bands = [cv2.imread(str(d / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED).astype(np.float32) for d in self.band_dirs]
        image = np.stack(bands, axis=-1)
        label = cv2.imread(str(self.label_dir / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED)
        if label.ndim == 3: label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)

        p02, p98 = np.percentile(image, 2), np.percentile(image, 98)
        image = np.clip(image, p02, p98)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        mask = np.zeros_like(label, dtype=np.int64)
        mask[label == 85] = 1; mask[label == 170] = 2; mask[label == 255] = 3

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug["image"].float(), aug["mask"].long()
        return torch.tensor(image.transpose(2,0,1)).float(), torch.tensor(mask).long()

class Wrapper(Dataset):
    def __init__(self, ds, t): self.ds, self.t = ds, t
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        img, mask = self.ds[i]
        img = img.numpy().transpose(1,2,0); mask = mask.numpy()
        res = self.t(image=img, mask=mask)
        return res['image'], res['mask'].long()

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5),
    A.GridDistortion(p=0.3),
    ToTensorV2(),
])
val_transform = A.Compose([ToTensorV2()])

full_ds = GlacierDataset(CONFIG['base_dir'], transform=train_transform)
val_len = int(len(full_ds)*0.2)
train_ds, val_ds = random_split(full_ds, [len(full_ds)-val_len, val_len], generator=torch.Generator().manual_seed(42))
val_ds.dataset.transform = val_transform

train_loader = DataLoader(Wrapper(train_ds, train_transform), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(Wrapper(val_ds, val_transform), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

# ==========================================
# 3. ARCHITECTURE: DEEPLAB RESNET50
# ==========================================

class SpikingASPP(nn.Module):
    def __init__(self, in_c, out_c, rates=[1, 6, 12, 18], mode="CNN"):
        super().__init__()
        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        def get_act():
            return snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)

        layers = []
        for rate in rates:
            layers.append(nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=rate, dilation=rate, bias=False),
                nn.BatchNorm2d(out_c),
                get_act()
            ))

        # Image Pooling Branch (1x1 Conv summary)
        layers.append(nn.Sequential(
            nn.Conv2d(in_c, out_c, 1, bias=False),
            nn.BatchNorm2d(out_c),
            get_act()
        ))

        self.branches = nn.ModuleList(layers)
        self.project = nn.Sequential(
            nn.Conv2d(len(layers) * out_c, out_c, 1, bias=False),
            nn.BatchNorm2d(out_c),
            get_act()
        )

    def forward(self, x):
        res = [branch(x) for branch in self.branches]
        res = torch.cat(res, dim=1)
        return self.project(res)

class ResNet50Backbone(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

        self.conv1 = nn.Conv2d(5, 64, 7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            self.conv1.weight[:, :3] = resnet.conv1.weight
            self.conv1.weight[:, 3:] = resnet.conv1.weight[:, :2]
        self.bn1 = resnet.bn1

        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        self.relu = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else resnet.relu
        self.maxpool = resnet.maxpool

        self.layer1 = self._convert(resnet.layer1, mode)
        self.layer2 = self._convert(resnet.layer2, mode)
        self.layer3 = self._convert(resnet.layer3, mode)
        self.layer4 = self._convert(resnet.layer4, mode)

    def _convert(self, block, mode):
        if mode == "CNN": return block
        layers = []
        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        for b in block:
            b.relu = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True)
            layers.append(b)
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x); x = self.bn1(x); x = self.relu(x)
        x = self.maxpool(x)
        l1 = self.layer1(x) # ResNet50 L1 is 256 channels
        l2 = self.layer2(l1)
        l3 = self.layer3(l2)
        l4 = self.layer4(l3) # ResNet50 L4 is 2048 channels
        return l1, l4

class UnifiedDeepLab(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        self.mode = mode
        print(f"⏳ Initializing DeepLabV3+ (ResNet50) {mode}...")

        self.backbone = ResNet50Backbone(mode)

        # ASPP takes 2048, outputs 256
        self.aspp = SpikingASPP(2048, 256, mode=mode)

        # Low Level: ResNet50 L1 is 256 channels. Project to 48.
        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        self.low_level_conv = nn.Sequential(
            nn.Conv2d(256, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(256 + 48, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True),

            nn.Conv2d(256, 4, 1)
        )

    def forward_single_step(self, x):
        low_level, high_level = self.backbone(x)
        high_level = self.aspp(high_level)
        high_level = F.interpolate(high_level, size=low_level.shape[2:], mode='bilinear', align_corners=False)
        low_level = self.low_level_conv(low_level)
        x = torch.cat([high_level, low_level], dim=1)
        x = self.decoder(x)
        return F.interpolate(x, scale_factor=4, mode='bilinear', align_corners=False)

    def forward(self, x):
        if self.mode == "SNN":
            spk_rec = []
            for step in range(CONFIG['time_steps']):
                out = self.forward_single_step(x)
                spk_rec.append(out)
            return torch.stack(spk_rec).mean(0)
        else:
            return self.forward_single_step(x)

# ==========================================
# 4. TRAINING ENGINE
# ==========================================
def manual_reset(model):
    for m in model.modules():
        if hasattr(m, "reset_mem"): m.reset_mem()

def save_vis(history, sample_vis, epoch, mode):
    plt.figure(figsize=(10, 5))
    plt.subplot(1,2,1); plt.plot(history['loss']); plt.title(f"{mode} Loss")
    plt.subplot(1,2,2); plt.plot(history['mcc']); plt.title(f"{mode} MCC")
    plt.savefig(f"{CONFIG['project_dir']}/{mode}_history.png"); plt.close()

    img, gt, pred = sample_vis
    rgb = img[[3,2,1]].transpose(1,2,0)
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-6)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(rgb); plt.title("Input")
    plt.subplot(1, 3, 2); plt.imshow(gt, cmap='nipy_spectral', interpolation='nearest'); plt.title("GT")
    plt.subplot(1, 3, 3); plt.imshow(pred, cmap='nipy_spectral', interpolation='nearest'); plt.title(f"{mode} Pred")
    plt.savefig(f"{CONFIG['project_dir']}/{mode}_sample.png"); plt.close()

def run_training(phase_name):
    torch.cuda.empty_cache()
    gc.collect()
    print(f"\n🚀 STARTING PHASE: {phase_name}")

    model = UnifiedDeepLab(mode=phase_name).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
    weights = torch.tensor([0.2, 1.0, 1.0, 3.0]).to(CONFIG['device'])
    criterion = nn.CrossEntropyLoss(weight=weights)
    scaler = GradScaler()

    best_mcc = -1.0
    history = {'loss': [], 'mcc': []}

    for epoch in range(CONFIG['epochs']):
        model.train()
        run_loss = 0

        loop = tqdm(train_loader, desc=f"{phase_name} Ep {epoch+1}")
        for imgs, masks in loop:
            imgs, masks = imgs.to(CONFIG['device']), masks.to(CONFIG['device'])

            if phase_name == "SNN": manual_reset(model)
            optimizer.zero_grad()

            if phase_name == "CNN":
                with autocast():
                    out = model(imgs)
                    loss = criterion(out, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(imgs)
                loss = criterion(out, masks)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            scheduler.step()
            run_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # Val
        model.eval()
        preds, targets = [], []
        sample_vis = None
        with torch.no_grad():
            for i, (imgs, masks) in enumerate(val_loader):
                imgs = imgs.to(CONFIG['device'])
                if phase_name == "SNN": manual_reset(model)
                if phase_name == "CNN":
                  with autocast():
                    out = model(imgs)
                else:
                  out = model(imgs)
                preds.append(out.argmax(1).cpu())
                targets.append(masks.cpu())
                if i==0: sample_vis = (imgs[0].cpu().numpy(), masks[0].cpu().numpy(), preds[-1][0].numpy())

        mcc = matthews_corrcoef(torch.cat(targets).numpy().flatten(), torch.cat(preds).numpy().flatten())
        history['mcc'].append(mcc)
        history['loss'].append(run_loss/len(train_loader))

        print(f"   ✅ Val MCC: {mcc:.4f}")
        save_vis(history, sample_vis, epoch+1, phase_name)

        if mcc > best_mcc:
            best_mcc = mcc
            torch.save(model.state_dict(), f"{CONFIG['project_dir']}/best_{phase_name}_DeepLab.pth")

    print(f"🏁 {phase_name} Finished. Best MCC: {best_mcc:.4f}")

if __name__ == "__main__":
    run_training("CNN")
    run_training("SNN")


🚀 STARTING PHASE: CNN
⏳ Initializing DeepLabV3+ (ResNet50) CNN...


/tmp/ipython-input-2299086882.py:270: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
CNN Ep 1:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2299086882.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
CNN Ep 1: 100%|██████████| 10/10 [00:04<00:00,  2.07it/s, loss=1.21]
/tmp/ipython-input-2299086882.py:3

   ✅ Val MCC: 0.2305


CNN Ep 2:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2299086882.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 2: 100%|██████████| 10/10 [00:05<00:00,  1.87it/s, loss=1.19]
/tmp/ipython-input-2299086882.py:313: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ✅ Val MCC: 0.3392


...

   ✅ Val MCC: 0.3212


SNN Ep 44: 100%|██████████| 10/10 [00:16<00:00,  1.60s/it, loss=0.634]


   ✅ Val MCC: 0.2959


SNN Ep 45: 100%|██████████| 10/10 [00:15<00:00,  1.60s/it, loss=0.95]


   ✅ Val MCC: 0.3123
🏁 SNN Finished. Best MCC: 0.3264


In [4]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import matthews_corrcoef
from torchvision import models
import snntorch as snn
from snntorch import surrogate
from snntorch import utils
import gc

CONFIG = {
    "base_dir": "/content/drive/MyDrive/glacier/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_DeepLab_EffNetB2",
    "model_type": "CNN",
    "remove_se": True, # Critical
    "time_steps": 6, "batch_size": 2, "lr": 1e-4, "epochs": 45,
    "beta": 0.9, "threshold": 0.5, "slope": 25,
    "num_workers": 2, "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

os.makedirs(CONFIG['project_dir'], exist_ok=True)
torch.cuda.empty_cache()
gc.collect()

# [INSERT DATASET CODE FROM PREVIOUS BLOCK HERE IF RUNNING SEPARATELY]
# For brevity, assuming Dataset class exists from Test 1.

# ==========================================
# DEEPLAB EFFICIENTNET-B2
# ==========================================
class SpikingASPP(nn.Module):
    def __init__(self, in_c, out_c, rates=[1, 6, 12, 18], mode="CNN"):
        super().__init__()
        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        def get_act():
            return snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)

        layers = []
        for rate in rates:
            layers.append(nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=rate, dilation=rate, bias=False),
                nn.BatchNorm2d(out_c),
                get_act()
            ))
        layers.append(nn.Sequential(nn.Conv2d(in_c, out_c, 1, bias=False), nn.BatchNorm2d(out_c), get_act()))
        self.branches = nn.ModuleList(layers)
        self.project = nn.Sequential(nn.Conv2d(len(layers) * out_c, out_c, 1, bias=False), nn.BatchNorm2d(out_c), get_act())

    def forward(self, x):
        res = [branch(x) for branch in self.branches]
        return self.project(torch.cat(res, dim=1))

class EfficientNetB2Backbone(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        effnet = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.DEFAULT)

        # Stem
        orig = effnet.features[0][0]
        effnet.features[0][0] = nn.Conv2d(5, 32, 3, 2, 1, bias=False)
        with torch.no_grad(): effnet.features[0][0].weight[:, :3] = orig.weight

        if mode == "SNN": self._convert_to_snn(effnet)
        self.features = effnet.features

        # Auto Detect
        self.low_ch, self.high_ch = self._get_channels()
        print(f"✅ B2 Detected: Low={self.low_ch}, High={self.high_ch}")

    def _convert_to_snn(self, model):
        for name, module in model.named_children():
            if isinstance(module, (nn.SiLU, nn.ReLU)):
                setattr(model, name, snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']), init_hidden=True))
            elif isinstance(module, models.efficientnet.SqueezeExcitation) and CONFIG['remove_se']:
                setattr(model, name, nn.Identity())
            elif isinstance(module, nn.Sigmoid):
                setattr(model, name, snn.Leaky(beta=CONFIG['beta'], threshold=0.5, spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']), init_hidden=True))
            else: self._convert_to_snn(module)

    def _get_channels(self):
        x = torch.randn(1, 5, 512, 512)
        # B2: Stem->256, Stage1->256, Stage2->128...
        # Low Level usually at 128x128 or 64x64. High Level is last.
        l1, l4 = None, None

        curr = x
        for i, layer in enumerate(self.features):
            curr = layer(curr)
            if curr.shape[2] == 128: l1 = curr.shape[1] # Capture at 128x128
        l4 = curr.shape[1]
        return l1, l4

    def forward(self, x):
        l1, l4 = None, None
        curr = x
        for i, layer in enumerate(self.features):
            curr = layer(curr)
            if curr.shape[2] == 128: l1 = curr # Keep updating until size drops
        l4 = curr
        return l1, l4

class UnifiedDeepLabEff(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        self.mode = mode
        print(f"⏳ Initializing DeepLabV3+ (EffNetB2) {mode}...")
        self.backbone = EfficientNetB2Backbone(mode)

        self.aspp = SpikingASPP(self.backbone.high_ch, 256, mode=mode)

        grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])
        self.low_level_conv = nn.Sequential(
            nn.Conv2d(self.backbone.low_ch, 48, 1, bias=False),
            nn.BatchNorm2d(48),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(256 + 48, 256, 3, padding=1, bias=False), nn.BatchNorm2d(256),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1, bias=False), nn.BatchNorm2d(256),
            snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True),
            nn.Conv2d(256, 4, 1)
        )

    def forward_single_step(self, x):
        low, high = self.backbone(x)
        high = self.aspp(high)
        high = F.interpolate(high, size=low.shape[2:], mode='bilinear', align_corners=False)
        low = self.low_level_conv(low)
        x = torch.cat([high, low], dim=1)
        x = self.decoder(x)
        return F.interpolate(x, scale_factor=4, mode='bilinear', align_corners=False)

    def forward(self, x):
        if self.mode == "SNN":
            spk_rec = []
            for step in range(CONFIG['time_steps']):
                out = self.forward_single_step(x)
                spk_rec.append(out)
            return torch.stack(spk_rec).mean(0)
        else:
            return self.forward_single_step(x)

# [INSERT TRAINING LOOP HERE - SAME AS TEST 1]
# For brevity, run Test 1 first, then this one.
if __name__ == "__main__":
    run_training("CNN")
    run_training("SNN")


🚀 STARTING PHASE: CNN
⏳ Initializing DeepLabV3+ (ResNet50) CNN...


/tmp/ipython-input-2299086882.py:270: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
CNN Ep 1:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2299086882.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 1: 100%|██████████| 10/10 [00:02<00:00,  3.44it/s, loss=1.35]
/tmp/ipython-input-2299086882.py:313: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ✅ Val MCC: 0.1060


CNN Ep 2:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-2299086882.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 2: 100%|██████████| 10/10 [00:02<00:00,  4.95it/s, loss=1.22]
/tmp/ipython-input-2299086882.py:313: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ✅ Val MCC: 0.2443


...

   ✅ Val MCC: 0.3254


SNN Ep 44: 100%|██████████| 10/10 [00:15<00:00,  1.60s/it, loss=0.952]


   ✅ Val MCC: 0.3168


SNN Ep 45: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it, loss=0.696]


   ✅ Val MCC: 0.2945
🏁 SNN Finished. Best MCC: 0.3429
